#### Generate map

In [4]:
import cv2
import random
import os
import json
from PIL import Image

# Paths
bg_dir = "/Users/wildanhidayat/Documents/GitHub/multi_agent_coordination_with_llms_and_vision_in_a_simulated_environment/assets/backgrounds"
obj_dir = "/Users/wildanhidayat/Documents/GitHub/multi_agent_coordination_with_llms_and_vision_in_a_simulated_environment/assets/objects"
save_dir = "/Users/wildanhidayat/Documents/GitHub/multi_agent_coordination_with_llms_and_vision_in_a_simulated_environment/assets/generated_maps"
os.makedirs(save_dir, exist_ok=True)

# Load background
bg_files = os.listdir(bg_dir)
bg_path = os.path.join(bg_dir, random.choice(bg_files))
background = Image.open(bg_path).convert("RGBA")

# Load object icons
obj_files = os.listdir(obj_dir)
selected_objects = random.sample(obj_files, 3)

# Initialize metadata file and name suffix
object_metadata = []
suffix = []

# Paste 3 random objects at random locations
for obj_file in selected_objects:
    obj_path = os.path.join(obj_dir, obj_file)
    obj_img = Image.open(obj_path).convert("RGBA")

    # Resize object (optional)
    obj_img = obj_img.resize((200, 200))

    # Random position
    x = random.randint(0, background.width - obj_img.width)
    y = random.randint(0, background.height - obj_img.height)

    # Paste object with transparency
    background.paste(obj_img, (x, y), obj_img)

    object_metadata.append({
        "object": os.path.splitext(obj_file)[0],  # e.g., 'tree'
        "position": {"x": x, "y": y},
        "size": {"width": obj_img.width, "height": obj_img.height}
    })

    suffix.append(os.path.splitext(obj_file)[0]+"_"+str(x)+"_"+str(y))

# Save the final map
name_suf = '_'.join([f for f in suffix])
map_name = f"map_%s" %(name_suf)

map_path = os.path.join(save_dir, f"{map_name}.png")
map_json_path = os.path.join(save_dir, f"{map_name}.json")

background.save(map_path)

with open(map_json_path, "w") as f:
    json.dump(object_metadata, f, indent=4)

print(f"Map generated: {map_path}")

Map generated: /Users/wildanhidayat/Documents/GitHub/multi_agent_coordination_with_llms_and_vision_in_a_simulated_environment/assets/generated_maps/map_cycle_1254_186_green_1709_396_people_1532_169.png


In [15]:
import cv2
import numpy as np
import os
import random
import json

ASSET_PATH = "/Users/wildanhidayat/Documents/GitHub/multi_agent_coordination_with_llms_and_vision_in_a_simulated_environment/assets/objects"  # Path to your object PNGs
MAP_SAVE_PATH = "/Users/wildanhidayat/Documents/GitHub/multi_agent_coordination_with_llms_and_vision_in_a_simulated_environment/assets/generated_maps/generated_map.png"
METADATA_SAVE_PATH = "/Users/wildanhidayat/Documents/GitHub/multi_agent_coordination_with_llms_and_vision_in_a_simulated_environment/assets/generated_maps/generated_map.json"

GRID_SIZE = (5, 5)
CELL_SIZE = (64, 64)  # Width, Height in pixels

def load_object_images(asset_path):
    object_files = [f for f in os.listdir(asset_path) if f.endswith(".png")]
    object_types = list(set(f.split(".")[0] for f in object_files))
    selected_types = random.sample(object_types, k=3)
    
    objects = {}
    for t in selected_types:
        img_path = os.path.join(asset_path, t + ".png")
        img = cv2.imread(img_path, cv2.IMREAD_UNCHANGED)
        if img is not None:
            img = cv2.resize(img, CELL_SIZE)
            objects[t] = img
    return objects

def paste_object(canvas, obj_img, x, y):
    if obj_img.shape[2] == 4:
        alpha = obj_img[:, :, 3] / 255.0
        for c in range(3):
            canvas[y:y+CELL_SIZE[1], x:x+CELL_SIZE[0], c] = (
                alpha * obj_img[:, :, c] +
                (1 - alpha) * canvas[y:y+CELL_SIZE[1], x:x+CELL_SIZE[0], c]
            )
    else:
        canvas[y:y+CELL_SIZE[1], x:x+CELL_SIZE[0]] = obj_img

def draw_grid(canvas):
    rows, cols = GRID_SIZE
    for r in range(1, rows):
        y = r * CELL_SIZE[1]
        cv2.line(canvas, (0, y), (canvas.shape[1], y), (200, 200, 200), 1)
    for c in range(1, cols):
        x = c * CELL_SIZE[0]
        cv2.line(canvas, (x, 0), (x, canvas.shape[0]), (200, 200, 200), 1)

def create_map():
    rows, cols = GRID_SIZE

    canvas = np.ones((rows * CELL_SIZE[1], cols * CELL_SIZE[0], 3), dtype=np.uint8) * 255
    draw_grid(canvas)

    metadata = {"start": None, "finish": None, "objects": []}

    available_cells = [(r, c) for r in range(rows) for c in range(cols)]

    # Randomly assign start and finish
    start = random.choice(available_cells)
    available_cells.remove(start)

    finish = random.choice(available_cells)
    available_cells.remove(finish)

    metadata["start"] = {"row": start[0], "col": start[1]}
    metadata["finish"] = {"row": finish[0], "col": finish[1]}

    # Draw start (green) and finish (red)
    start_img = np.full((*CELL_SIZE, 3), (0, 0, 0), dtype=np.uint8)
    finish_img = np.full((*CELL_SIZE, 3), (255, 0, 0), dtype=np.uint8)

    paste_object(canvas, start_img, start[1] * CELL_SIZE[0], start[0] * CELL_SIZE[1])
    paste_object(canvas, finish_img, finish[1] * CELL_SIZE[0], finish[0] * CELL_SIZE[1])

    # Load 3 random object types
    objects = load_object_images(ASSET_PATH)

    for obj_type, obj_img in objects.items():
        cell = random.choice(available_cells)
        available_cells.remove(cell)
        paste_object(canvas, obj_img, cell[1] * CELL_SIZE[0], cell[0] * CELL_SIZE[1])
        metadata["objects"].append({"type": obj_type, "row": cell[0], "col": cell[1]})

    # Save map and metadata
    os.makedirs("maps", exist_ok=True)
    cv2.imwrite(MAP_SAVE_PATH, canvas)
    with open(METADATA_SAVE_PATH, "w") as f:
        json.dump(metadata, f, indent=2)

    print("Map and metadata saved!")

if __name__ == "__main__":
    create_map()

Map and metadata saved!


#### HHHH

In [1]:
import cv2
import os
import random

def list_image_files(folder_path, extensions={'.png', '.jpg', '.jpeg'}):
    return [
        f for f in os.listdir(folder_path)
        if os.path.isfile(os.path.join(folder_path, f)) and os.path.splitext(f)[1].lower() in extensions
    ]

def load_and_display_map(map_path):
    img = cv2.imread(map_path)
    if img is None:
        raise ValueError(f"Cannot load image at {map_path}")
    cv2.imshow("Simulation Environment", img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

if __name__ == "__main__":
    map_folder = "/Users/wildanhidayat/Documents/GitHub/multi_agent_coordination_with_llms_and_vision_in_a_simulated_environment/assets/generated_maps"
    
    # Import all map file name
    map_images = list_image_files(map_folder)
    
    # Choose one map randomly
    map_file = random.choice(map_images)  

    # Define the map path
    full_path = os.path.join(map_folder, map_file)

    # Load and display the selected map
    load_and_display_map(full_path)

IndexError: list index out of range

In [5]:
import random
random.choice(object_images)

'map_car_40_46_tree_207_217_cycle_1182_479.png'